# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
df=pd.read_csv("city_data")
df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vaini,-21.2000,-175.2000,78.96,73,20,11.50,TO,1635034722
1,Ushuaia,-54.8000,-68.3000,35.26,80,90,18.41,AR,1635034628
2,Bredasdorp,-34.5322,20.0403,49.32,87,24,4.61,ZA,1635034724
3,Peñarroya-Pueblonuevo,38.3000,-5.2667,57.45,48,43,2.10,ES,1635034937
4,Puerto Ayora,-0.7393,-90.3518,69.26,87,90,9.22,EC,1635034717
...,...,...,...,...,...,...,...,...,...
564,Samaná,19.2056,-69.3369,78.37,82,28,7.25,DO,1635035079
565,Abu Dhabi,24.4667,54.3667,84.22,65,20,3.44,AE,1635034508
566,Wagga Wagga,-35.1167,147.3667,64.60,63,40,9.22,AU,1635035079
567,Barra do Garças,-15.8900,-52.2567,81.41,57,100,5.21,BR,1635034878


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:

#configure gmaps
gmaps.configure(api_key=g_key)

In [21]:
locations = df[['Lat','Lng']]
humidity = df['Humidity']

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity))
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
#A max temperature lower than 80 degrees but higher than 70.
temp_df = df.loc[(df['Max Temp']<80) & (df['Max Temp']>70)]

#Wind speed less than 10 mph.
wind_df = temp_df.loc[(temp_df['Wind Speed']<10)]

#Zero cloudiness
cloud_df = wind_df.loc[(wind_df['Cloudiness']==0)]
cloud_df



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
142,Salalah,17.0151,54.0924,73.49,64,0,5.75,OM,1635034771
289,Santa María del Oro,25.9333,-105.3667,72.68,30,0,4.72,MX,1635035006
396,Lohāru,28.4500,75.8167,72.18,56,0,5.59,IN,1635035033
486,Sur,22.5667,59.5289,79.84,65,0,9.75,OM,1635034842
519,Aswān,24.0934,32.9070,72.03,41,0,9.53,EG,1635035066


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df=cloud_df

In [33]:
hotel_df["Hotel Name"]=""
hotel_df

<ipython-input-33-4b7c85aad61d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
142,Salalah,17.0151,54.0924,73.49,64,0,5.75,OM,1635034771,
289,Santa María del Oro,25.9333,-105.3667,72.68,30,0,4.72,MX,1635035006,
396,Lohāru,28.4500,75.8167,72.18,56,0,5.59,IN,1635035033,
486,Sur,22.5667,59.5289,79.84,65,0,9.75,OM,1635034842,
519,Aswān,24.0934,32.9070,72.03,41,0,9.53,EG,1635035066,


In [40]:
hotel_df.reset_index()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,142,Salalah,17.0151,54.0924,73.49,64,0,5.75,OM,1635034771,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
1,289,Santa María del Oro,25.9333,-105.3667,72.68,30,0,4.72,MX,1635035006,Hotel Palacio Del Sol
2,396,Lohāru,28.4500,75.8167,72.18,56,0,5.59,IN,1635035033,Sainik Guest House
3,486,Sur,22.5667,59.5289,79.84,65,0,9.75,OM,1635034842,Sur Plaza Hotel
4,519,Aswān,24.0934,32.9070,72.03,41,0,9.53,EG,1635035066,Nuba Nile


In [39]:

target_radius = 5000
target_type = 'lodging'

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

/Users/alexanderlorin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [41]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
142,Salalah,17.0151,54.0924,73.49,64,0,5.75,OM,1635034771,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
289,Santa María del Oro,25.9333,-105.3667,72.68,30,0,4.72,MX,1635035006,Hotel Palacio Del Sol
396,Lohāru,28.4500,75.8167,72.18,56,0,5.59,IN,1635035033,Sainik Guest House
486,Sur,22.5667,59.5289,79.84,65,0,9.75,OM,1635034842,Sur Plaza Hotel
519,Aswān,24.0934,32.9070,72.03,41,0,9.53,EG,1635035066,Nuba Nile


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations, 
    info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)
fig.add_layer(hotel_layer)




# Display figure
fig



Figure(layout=FigureLayout(height='420px'))